In [34]:

from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.dates import DateFormatter


def calculate(df, element, func):
    res = df[df['Element'] == element].groupby('Period').agg({'Data_Value': func})
    res.index = res.index.map(lambda d: pd.to_datetime(d, format='%m%d'))
    return res

binsize = 400
hashid = 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89'
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df['Data_Value'] = df['Data_Value']/10
df['Date'] = pd.to_datetime(df['Date'])
df = df[~((df['Date'].dt.month == 2) & (df['Date'].dt.day == 29))]
df['Period']  = df['Date'].apply(lambda d: d.strftime('%m%d'))


df2015 = df[df['Date'].dt.year == 2015]
dfHist = df[df['Date'].dt.year < 2015]

# Data_Value
dfHist_min = calculate(dfHist, 'TMIN', np.min)
dfHist_max = calculate(dfHist, 'TMAX', np.max) 

df2015_min = calculate(df2015, 'TMIN', np.min)
df2015_max = calculate(df2015, 'TMAX', np.max) 

df2015_below_min = pd.merge(df2015_min, dfHist_min, left_index = True, right_index=True)
df2015_below_min['below'] = df2015_below_min.apply(
    lambda r : r['Data_Value_x'] if r['Data_Value_x'] < r['Data_Value_y'] else None, axis=1)
df2015_below_min = df2015_below_min[['below']]

df2015_above_max = pd.merge(df2015_max, dfHist_max, left_index = True, right_index=True)
df2015_above_max['above'] = df2015_above_max.apply(
    lambda r : r['Data_Value_x'] if r['Data_Value_x'] > r['Data_Value_y'] else None, axis=1)
df2015_above_max = df2015_above_max[['above']]

plt.figure()
plt.title('Temperatures on Ann Arbor, MI (2005-2015)')
plt.ylabel('℃ degrees')
plt.plot(dfHist_max, '-', color='lightcoral')
plt.plot(dfHist_min, '-', color='tab:blue')
plt.scatter(df2015_above_max.index, df2015_above_max['above'].values, color='red', marker='.', s=100)
plt.scatter(df2015_below_min.index, df2015_below_min['below'].values, color='blue', marker='.', s=100)

plt.gca().fill_between(dfHist_max.index, 
                       dfHist_min['Data_Value'].values, dfHist_max['Data_Value'].values, 
                      facecolor='grey', alpha=0.25)


x = plt.gca().xaxis

plt.gca().set_xticks(list(filter(lambda i:i.day==15, dfHist_max.index)))
x.set_major_formatter(DateFormatter('%b'))

for item in x.get_ticklabels():
    item.set_rotation(45)
    
for spine in plt.gca().spines.values():
    spine.set_visible(False) 
    
plt.tick_params(top='off', bottom='off', left='off', right='off', labelleft='on', labelbottom='on')
    

plt.legend(['2005-2014 record highs','2005-2014 record lows','2015 record high','2015 record low'], loc='best')



plt.savefig('assignment2.png', transparent=False)
